<a href="https://colab.research.google.com/github/jngadiub/ML_course_Pavia_23_WIP/blob/main/PartT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jet Tagging with Set Transformer

In this notebook we will see an implementation of the Transformer architecture for sets applied to the jet tagging task. For *sets* it is meant here a point cloud, i.e. a set of nodes without edges. We will instead use Multi-Head Attention to learn which nodes (or particles) have strong pair-wise interaction.

The architecture was introduced by [J. Lee at al. (ICML 2019)](https://arxiv.org/abs/1810.00825) --  specifically designed to model interactions among elements in the input set without pre-defined edges. The model consists of an encoder and a decoder, both of which rely on attention mechanisms, as in the original Transformer implementation [by Vaswani](https://arxiv.org/abs/1706.03762). The main difference is that positional encoding is removed plus some other low level adaptions.

We will use tensorflow for this implementation.

Before you start, choose GPU as a hardware accelerator for this notebook. To do this first go to Edit -> Notebook Settings -> Choose GPU as a hardware accelerator.

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
import h5py
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import roc_curve, auc, confusion_matrix
from networks import TransformerNet,RegNet,ClassNet,OutNet,PoolingByMultiHeadAttention

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [22]:
# let's open the file
data_dir = 'Data-MLtutorial/JetDataset/'
fileIN = data_dir+'jetImage_7_100p_30000_40000.h5'
f = h5py.File(fileIN,"r")
# and see what it contains
print(list(f.keys()))

['jetConstituentList', 'jetFeatureNames', 'jetImage', 'jetImageECAL', 'jetImageHCAL', 'jets', 'particleFeatureNames']


In [ ]:
target_onehot = np.array([])
jetList = np.array([])
jetImages = np.array([])
target_reg = np.array([])
features_names = dict()
datafiles = ['jetImage_7_100p_0_10000.h5',
             'jetImage_7_100p_10000_20000.h5',
             'jetImage_7_100p_30000_40000.h5',
             'jetImage_7_100p_40000_50000.h5',
             'jetImage_7_100p_50000_60000.h5',
             'jetImage_7_100p_60000_70000.h5',
             'jetImage_7_100p_70000_80000.h5',
             'jetImage_7_100p_80000_90000.h5'
            ]
for i_f,fileIN in enumerate(datafiles):
    print("Appending %s" %fileIN)
    f = h5py.File(data_dir + fileIN)
    jetList_file = np.array(f.get("jetConstituentList"))
    target_file = np.array(f.get('jets')[0:,-6:-1])
    mass = np.array(f.get("jets")[0:,3])    
    jetImages_file = np.array(f.get('jetImage'))
    jetList = np.concatenate([jetList, jetList_file], axis=0) if jetList.size else jetList_file
    target_onehot = np.concatenate([target_onehot, target_file], axis=0) if target_onehot.size else target_file
    jetImages = np.concatenate([jetImages, jetImages_file], axis=0) if jetImages.size else jetImages_file
    target_reg = np.concatenate([target_reg,mass],axis=0)
    del jetList_file, target_file, jetImages_file, mass
    #save particles/nodes features names and their indecies in a dictionary
    if i_f==0:
      for feat_idx,feat_name in enumerate(list(f['particleFeatureNames'])[:-1]):
        features_names[feat_name.decode("utf-8").replace('j1_','')] = feat_idx
    f.close()

Appending jetImage_7_100p_0_10000.h5


/tmp/ipykernel_151683/3009411415.py:17: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(data_dir + fileIN)


Appending jetImage_7_100p_10000_20000.h5
Appending jetImage_7_100p_30000_40000.h5
Appending jetImage_7_100p_40000_50000.h5
Appending jetImage_7_100p_50000_60000.h5
Appending jetImage_7_100p_60000_70000.h5
Appending jetImage_7_100p_70000_80000.h5


In [ ]:
target = np.argmax(target_onehot, axis=1)
num_classes = len(np.unique(target))
label_names= ["gluon", "quark", "W", "Z", "top"]

In [ ]:
print('Jets shape : ',jetList.shape)
print('Target/Labels shape : ',target.shape)
print('Particles/Nodes features : ',list(features_names.keys()))

In [ ]:
labelCat = ["gluon", "quark", "W", "Z", "top"]
def plot_features(features):
    for i,f in enumerate(features.keys()):
        for l in range(num_classes):
            idxs  = np.where([target==l])
            plt.hist(jetList[idxs,:,i].flatten(),50, density=True, histtype='step', fill=False, linewidth=1.5,label=labelCat[l])
        plt.yscale('log')    
        plt.legend(fontsize=12, frameon=False)
        plt.xlabel(f)
        plt.ylabel('Prob. Density (a.u.)', fontsize=15)
        plt.show()
#plot_features(features_names)

In [ ]:
#features_to_consider = 'etarel,phirel,pt,e,ptrel,erel,deltaR'.split(',')
#features_idx = [features_names[name] for name in features_to_consider]
#jetList = jetList[:,:,features_idx]
max_mass = np.max(target_reg)
target_reg /= max_mass
print(jetList.shape)

In [ ]:
batch_size=64
learning_rate=0.001
epochs=20

In [ ]:
num_heads = 8
hidden_dimensions = 64

t_net = TransformerNet(num_heads=num_heads, hidden_units=hidden_dimensions)
r_net = RegNet(hidden_dimensions)
c_net = ClassNet(hidden_dimensions)

inputs = keras.Input(shape=(100,16), name='input')
output = layers.TimeDistributed(layers.Dense(hidden_dimensions))(inputs)
output = t_net(output)
# output = PoolingByMultiHeadAttention(num_heads=num_heads,hidden_units=hidden_dimensions)(output)
output = layers.Lambda(lambda y: tf.reduce_sum(y, axis=1))(output)
output = [r_net(output), c_net(output)]
# output = o_net(output)

model = keras.models.Model(inputs=inputs, outputs=output)
model.summary()

In [ ]:
model.compile(
    loss=["mse", keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    loss_weights=[0.5,0.5])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_class, y_val_class, y_train_reg, y_val_reg, y_train_onehot, y_val_onehot = train_test_split(jetList, target,target_reg, target_onehot, test_size=0.1, shuffle=True)
print(X_train.shape, X_val.shape, y_train_class.shape, y_val_class.shape, y_train_reg.shape, y_val_reg.shape)
del jetList, target, target_onehot

In [ ]:
history = model.fit(x=X_train, 
                    y=[y_train_reg,y_train_class], 
                    validation_data=(X_val,[y_val_reg,y_val_class]), 
                    batch_size=batch_size, 
                    epochs=epochs,
                    verbose=1,
                    callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
                                  TerminateOnNaN()]
                   )

We can now plot the validation and training loss evolution over the epochs:

In [ ]:
keys = ["loss","Reg_loss","Class_loss","Class_sparse_categorical_accuracy"]
fig,axes = plt.subplots(2,2, figsize=(20,20))
for ax, key in zip(axes.flat, keys):
    ax.plot([i for i in range(len(history.history[key]))],history.history[key],label=key)
    ax.plot([i for i in range(len(history.history[key]))],history.history["val_"+key],label="val_"+key)
    ax.legend()

fig.show()

Now we finally evaluate the performance by plotting the ROC curves for the different classes:

In [ ]:
%matplotlib inline
predict_val = tf.nn.softmax(model.predict(X_val)[1]) # index 0 is the regression predictio
df = pd.DataFrame()
fpr = {}
tpr = {}
auc1 = {}

plt.figure()
for i, label in enumerate(label_names):

        df[label] = y_val_onehot[:,i]
        df[label + '_pred'] = predict_val[:,i]

        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])

        auc1[label] = auc(fpr[label], tpr[label])

        plt.plot(fpr[label],tpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
#plt.semilogy()
plt.ylabel("sig. efficiency")
plt.xlabel("bkg. mistag rate")
plt.ylim(0.000001,1)
plt.grid(True)
plt.legend(loc='lower right')
plt.show()

In [ ]:
predict_reg = model.predict(X_val)[0].flatten()
plt.hist(((predict_reg-y_val_reg)/y_val_reg)*max_mass,50,(-200,200),density=True,histtype="step")
plt.xlabel("Predicted  - True  / True ")
plt.ylabel('Prob. Density (a.u.)', fontsize=15)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_mat = confusion_matrix(np.argmax(predict_val,axis=1),y_val_class)
plt.figure(figsize=(8, 6))
plt.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, range(num_classes))
plt.yticks(tick_marks, range(num_classes))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')